In [1]:
import copy
from math import fabs

class ProblemState:
    
    __visited_states = []
    
    def __init__(self, parent, state):
        self.parent = parent
        self.state = state
        self.childs = []
    
    def __equal_states(first_state, second_state):
        for i in range(3):
            for j in range(3):
                if first_state[i][j] != second_state[i][j]:
                    return False
        return True
    
    def __has_visited(some_state):
        for state in ProblemState.__visited_states:
            if ProblemState.__equal_states(state, some_state):
                return True
        return False
        
    def __are_neighbors(self, position_1, position_2):
        condition_1 = position_1[0] == position_2[0] and fabs(position_1[1] - position_2[1]) == 1
        condition_2 = position_1[1] == position_2[1] and fabs(position_1[0] - position_2[0]) == 1
        return condition_1 or condition_2
        
    def __find_zero_position(self):
        i = 0
        j = 0
        zero_position = (i, j)
        for line in self.state:
            for element in line:
                if element == 0:
                    zero_position = (i, j)
                j += 1
            j = 0
            i += 1
        return zero_position
        
    def is_solution(self):
        i = 0
        j = 0
        counter = 1
        for line in self.state:
            for element in line:
                if (i != 2 or j != 2) and element != counter:
                    return False
                j += 1
                counter += 1
            j = 0
            i += 1
        return True  

    def __next_state(self, position_1, position_2):
        next_state = copy.deepcopy(self.state)
        value_1 = next_state[position_1[0]][position_1[1]]
        value_2 = next_state[position_2[0]][position_2[1]]
        next_state[position_1[0]][position_1[1]] = value_2
        next_state[position_2[0]][position_2[1]] = value_1
        return next_state

    def generate_childs(self):
        if self.is_solution() == False:
            if ProblemState.__has_visited(self.state) == False:
                ProblemState.__visited_states.append(self.state)
            zero_position = self.__find_zero_position()
            for i in range(3):
                for j in range(3):
                    if self.__are_neighbors(zero_position, (i, j)):
                        next_state = self.__next_state(zero_position, (i, j))
                        if ProblemState.__has_visited(next_state) == False:
                            self.childs.append(ProblemState(self, next_state))
                            ProblemState.__visited_states.append(next_state)
                            
    def wrong_pieces_count(self):
        i = 0
        j = 0
        counter = 1
        wrong_pieces = 0
        if self.state[2][2] != 0:
            wrong_pieces += 1
        for line in self.state:
            for element in line:
                if (i != 2 or j != 2) and element != counter:
                    wrong_pieces += 1
                j += 1
                counter += 1
            j = 0
            i += 1
        return wrong_pieces

In [2]:
class Problem:
    
    __visited_states = []
    
    def __init__(self, initial_config):
        self.root_state = ProblemState(None, initial_config)
        
    def __run_bfs(self, states):
        if len(Problem.__visited_states) >= 100:
            self.__print_no_solution("breadth-first search")
            return
        else:
            childs = []
            for state_collection in states:
                for state in state_collection:
                    Problem.__visited_states.append(state.state)
                    if state.is_solution():
                        self.__print_solution("breadth-first search")
                        return    
                    state.generate_childs()
                    childs.append(state.childs)
            if not childs:
                self.__print_no_solution("breadth-first search")
                return
            else:
                self.__run_bfs(childs)
                
    def __run_a_star(self, states):
        if len(Problem.__visited_states) >= 100:
            self.__print_no_solution("A-star")
            return
        else:
            childs = []
            for state_collection in states:
                for state in state_collection:
                    Problem.__visited_states.append(state.state)
                    if state.is_solution():
                        self.__print_solution("A-star")
                        return    
                    state.generate_childs()
                    if len(state.childs) != 0:
                        best_child = state.childs[0]
                        for child in state.childs:
                            if child.wrong_pieces_count() < best_child.wrong_pieces_count():
                                best_child = child
                        childs.append([best_child])
            if not childs:
                self.__print_no_solution("A-star")
                return
            else:
                self.__run_a_star(childs)
                
    def __print_solution(self, algorithm):
        print(algorithm)
        print("estados visitados: " + str(len(Problem.__visited_states)))
        for state in Problem.__visited_states:
            print(state)
            
    def __print_no_solution(self, algorithm):
        print(algorithm)
        print("no solution")
        print("estados visitados: " + str(len(Problem.__visited_states)))
                    
    def run(self):
        self.__run_bfs([[self.root_state]])
        Problem.__visited_states = []
        print()
        self.__run_a_star([[self.root_state]])

In [3]:
initial_config = [[1, 0, 2], [4, 5, 3], [7, 8, 6]]
problem = Problem(initial_config)
problem.run()

breadth-first search
estados visitados: 13
[[1, 0, 2], [4, 5, 3], [7, 8, 6]]
[[0, 1, 2], [4, 5, 3], [7, 8, 6]]
[[1, 2, 0], [4, 5, 3], [7, 8, 6]]
[[1, 5, 2], [4, 0, 3], [7, 8, 6]]
[[4, 1, 2], [0, 5, 3], [7, 8, 6]]
[[1, 2, 3], [4, 5, 0], [7, 8, 6]]
[[1, 5, 2], [0, 4, 3], [7, 8, 6]]
[[1, 5, 2], [4, 3, 0], [7, 8, 6]]
[[1, 5, 2], [4, 8, 3], [7, 0, 6]]
[[4, 1, 2], [5, 0, 3], [7, 8, 6]]
[[4, 1, 2], [7, 5, 3], [0, 8, 6]]
[[1, 2, 3], [4, 0, 5], [7, 8, 6]]
[[1, 2, 3], [4, 5, 6], [7, 8, 0]]

A-star
estados visitados: 4
[[1, 0, 2], [4, 5, 3], [7, 8, 6]]
[[1, 2, 0], [4, 5, 3], [7, 8, 6]]
[[1, 2, 3], [4, 5, 0], [7, 8, 6]]
[[1, 2, 3], [4, 5, 6], [7, 8, 0]]
